![](./figures/Logo.PNG)

Please click the <span>&#x23E9;</span> button to run all cells before you start working on the notebook ...

## In this part of the tutorial, you will
* compare local and global sensitivity analysis

---

# 5 a - Local and Regional Sensitivity Analysis

---

## 1 Loading Catchment Dat

In [1]:
import sys
sys.path.append('src/')
import HBV
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import RSA_thres as RSA           # module to perform RSA with threshold
import plot_functions
import sampling
from ipywidgets import interact, Dropdown, FloatSlider

In [2]:
def rmse(obs, sim, spinup=365):
    obs = obs[spinup:]
    sim = sim[spinup:]
    return np.sqrt(np.mean(np.square(np.subtract(obs, sim))))

def abs_bias(obs, sim, spinup=365):
    obs = obs[spinup:]
    sim = sim[spinup:]
    return np.mean(np.abs(sim - obs))

def hbv(par, precip, temp, evap):
    # Run HBV snow routine
    p_s, _, _ = HBV.snow_routine(par[:4], temp, precip)
    # Run HBV runoff simulation
    Case = 1 # for now we assume that the preferred path in the upper zone is runoff (Case = 1), it can be set to percolation (Case = 2)
    ini = np.array([0,0,0]) # initial state
    runoff_sim, _, _ = HBV.hbv_sim(par[4:], p_s, evap, Case, ini)
    return runoff_sim

In [3]:
# DO NOT ALTER! code to select the catchment

catchment_names = ["Medina River, TX, USA", "Siletz River, OR, USA", "Trout River, BC, Canada"]
dropdown = Dropdown(
    options=catchment_names,
    value=catchment_names[0],
    description='Catchment:',
    disabled=False
)

display(dropdown)

Dropdown(description='Catchment:', options=('Medina River, TX, USA', 'Siletz River, OR, USA', 'Trout River, BC…

In [4]:
# read catchment data
catchment_name = dropdown.value
file_dic = {catchment_names[0]: "camels_08178880", catchment_names[1]: "camels_14305500", catchment_names[2]: "hysets_10BE007"}
df_obs = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")

# correctly load the date and restrict analysis to one year
df_obs.date = pd.to_datetime(df_obs['date'], format='%Y-%m-%d')
start_date  = '2003-01-01' # the first year is used as spinup
end_date    = '2008-12-30'

# Index frame by date
df_obs.set_index('date', inplace=True)
# Select time frame
df_obs = df_obs[start_date:end_date]
# Reformat the date for plotting
df_obs["date"] = df_obs.index.map(lambda s: s.strftime('%b-%d-%y'))
# Reindex
df_obs = df_obs.reset_index(drop=True)
# Select snow, precip, PET, streamflow and T
df_obs = df_obs[["snow_depth_water_equivalent_mean", "total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean", "date"]]
# Rename variables
df_obs.columns = ["Snow [mm/day]", "P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]", "Date"]

# load calibrated parameters
params_calibrated = pd.read_csv("./data/calibrated_parameters - HBV.csv")
params_calibrated = params_calibrated[(params_calibrated.catchment_name == catchment_name) & (params_calibrated.objective_function == "nse")] # use only this catchment and the rmse parameters
params_calibrated = params_calibrated.iloc[0,3:].values

## 2 Global Sensitivity Analysis with AAT Sampling and Difference of CDFs

In [5]:
N = 250

param_names = ["Ts", "CFMAX", "CFR", "CWH", "BETA", "LP", "FC", "PERC", "K0", "K1", "K2", "UZL", "MAXBAS"]
lower       = np.array([-3, 0, 0, 0, 0, 0.3, 1, 0, 0.05, 0.01, 0.005, 0, 1])
upper       = np.array([3, 20, 1, 0.8, 7, 1, 2000, 100, 2, 1, 0.1, 100, 6])

# take N samples using a latin-hypercube strategy from the whole paramters space
np.random.seed(1234)
samples = sampling.AAT_sampling("lhs", len(param_names), sp.stats.uniform, np.array([lower, upper - lower]).T.tolist(), N=N)

# evaluate HBV for these parameters
errors = np.zeros((N, 2))
for i in range(N):
    Q_sim = hbv(samples[i], df_obs["P [mm/day]"], df_obs["T [C]"], df_obs["PET [mm/day]"])
    errors[i, 0] = abs_bias(df_obs["Q [mm/day]"], Q_sim) # calculate bias for this paramter set
    errors[i, 1] =     rmse(df_obs["Q [mm/day]"], Q_sim) # calculate rmse for this paramter set

In [6]:
@interact(bias=FloatSlider(value=np.quantile(errors[:,0], 0.2), min=errors[:,0].min(), max=errors[:,0].max(), step=0.005, description="|Bias|"), rmse=FloatSlider(value=np.quantile(errors[:,1], 0.2), min=errors[:,1].min(), max=errors[:,1].max(), step=0.005, description="RMSE"))
def plot_behavioral(bias, rmse):
    
    # grid layout for plot
    fig = plt.figure(figsize=(4*3, 3*3))
    gsl = fig.add_gridspec(3, 4)

    # subplot for the sensitivity values
    ax_sens = fig.add_subplot(gsl[0,:])
    ax_sens.set_xticks(range(13), param_names)
    ax_sens.set_title("Paramter Importance Based on Max. Diff. in CDF")
    ax_sens.set_ylabel("Max. Diff. in CDF")
    
    for i, metric in enumerate(["|Bias|", "RMSE"]):

        # which paramters are behavioral
        behavioral = (errors[:,i] <= [bias, rmse][i])

        # calculate experimental CDFs for the behavioral and non-behavioral parameters
        ecdfs = np.array([[sp.stats.ecdf(samples[mask, k]).cdf for k in range(13)] for mask in [behavioral, ~behavioral]])
        
        for j, param in enumerate(["CFMAX", "BETA", "FC", "K2"]):
    
            # index of the parameter value
            k = param_names.index(param)
            x = np.linspace(lower[k], upper[k], 100)
            
            ax = fig.add_subplot(gsl[i + 1, j])
            if j == 0: ax.set_title(f"Behavioral using {metric}")
            if j == 0: ax.set_ylabel("CDF")
            if j != 0: ax.set_yticklabels([])
            ax.set_xlabel(param)
            ax.set_xlim(samples[:,k].min(), samples[:,k].max())
            ax.set_ylim([-0.05, 1.05])
    
            # plot cdf of behavioral
            if behavioral.sum() > 1:
                ax.step(x, [ecdfs[0, k].evaluate(xi) for xi in x], where="mid", color="blue", label=f"{np.mean(behavioral):2.0%} Behavioral")
    
            # plot cdf of nonbehavioral
            if behavioral.sum() < len(behavioral):
                ax.step(x, [ecdfs[1, k].evaluate(xi) for xi in x], where="mid", color="red", label=f"{1 - np.mean(behavioral):2.0%} Nonbehavioral")
    
            # plot maximum difference when both curves are given
            if 1 < behavioral.sum() < len(behavioral):
                argmax = np.argmax(np.abs([ecdfs[0, k].evaluate(xi) - ecdfs[1, k].evaluate(xi) for xi in x]))
                rect = ax.add_patch(plt.matplotlib.patches.Rectangle((x[argmax - 1], ecdfs[0, k].evaluate(x[argmax])), x[argmax + 1] - x[argmax - 1], ecdfs[1, k].evaluate(x[argmax]) - ecdfs[0, k].evaluate(x[argmax])))
                rect.set(color=f"C{k}", hatch=None if i==0 else "//", edgecolor="white")
            
            # rugplot of the behavioral paramters
            ax.plot(samples[ behavioral,k], -0.025 + np.zeros_like(samples[ behavioral,k]), '|', color='blue', alpha=0.2) 
            ax.plot(samples[~behavioral,k],  1.025 + np.zeros_like(samples[~behavioral,k]), '|', color='red',  alpha=0.2)

            if j == 3: ax.legend()
    
        for j, param in enumerate(param_names):
    
            # index of the paramter value
            k = param_names.index(param)
            x = np.linspace(lower[k], upper[k], 100)
    
            if 1 < behavioral.sum() < len(behavioral):
                diffmax = np.max(np.abs([ecdfs[0, k].evaluate(xi) - ecdfs[1, k].evaluate(xi) for xi in x]))
                ax_sens.bar(j - 0.17 if i == 0 else j + 0.17, diffmax, width=0.3, color=f"C{k}", hatch=None if i == 0 else "//", edgecolor="white", label=None if param != "BETA" else ["|Bias|", "RMSE"][i])


    ax_sens.legend()    
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.3698953205832412, description='|Bias|', max=1.603661667104514, min=0…

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
    <h4><span>&#129300 </span>Task I: Global Sensitivity Analysis</h4>
    <p>
        Similar to what we did in the last tutorial, we have again ran HBV for parameter combinations samples from the whole parameter space using a LHS strategy. These runs are again classified as behavioral when the RMSE or |Bias| are below a certain threshold, which you can set. In constrast to last weeks tutorial, the runs do not need to fulfill both conditions, meaning that we find different behavioral parameter sets for the two metrics.
    </p>
    <p>
        The above plot shows the empirical cumulative density function (CDF) of the behavioral and nonbehavioral parameters for the two metrics. Based on the maximum absolute difference between the CDFs for behavioral and nonbehavioral curves we can define a measure of parameter importance. In other words: if a paramter is important for the evaluation of the model run, it should influence which parameter sets are classificed as behavioral. The first row in the plot shows this measure for each parameter and metric.
    </p>
    <ol>
        <li>What are benefits and shortcomings to define "importance" using this approach?</li>
        <li>Which parameters are important? Are there differences between metrics?</li>
        <li>How do the thresholds influence parameter importance? How would you choose a limit for screening important paramters?</li>
    </ol>
</div>

_PUT YOUR ANSWER HERE_

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
</div>

## 3 Local Sensitivity Analysis with OAT and Derivatives

In [7]:
@interact(r=FloatSlider(value=0.1, min=0.01, max=0.2, step=0.01, readout_format=".0%", description=r"$\Delta x$"))
def plot_OAT(r=0.1):
    
    # grid layout for plot
    fig = plt.figure(figsize=(4*3, 3*3))
    gsl = fig.add_gridspec(3, 4)

    # subplot for the sensitivity values
    ax_sens = fig.add_subplot(gsl[0,:])
    ax_sens.set_xticks(range(13), param_names)
    ax_sens.set_title("Paramter Importance Based on Local Relative Derivative")
    ax_sens.set_ylabel("Local Relative Derivative")
    
    for i, metric in enumerate(["|Bias|", "RMSE"]):
        
        for k, param in enumerate(param_names):

            # calibrated parameter set
            csample = params_calibrated.copy()
            Q_sim   = hbv(csample, df_obs["P [mm/day]"], df_obs["T [C]"], df_obs["PET [mm/day]"])
            csample = csample
            cmetric = [abs_bias, rmse][i](df_obs["Q [mm/day]"], Q_sim)
            cmetric = cmetric

            # percentage change to lower values
            lsample    = params_calibrated.copy()
            lsample[k] = lsample[k]*(1 - r)
            Q_sim      = hbv(lsample, df_obs["P [mm/day]"], df_obs["T [C]"], df_obs["PET [mm/day]"])
            lsample    = lsample/csample
            lmetric    = [abs_bias, rmse][i](df_obs["Q [mm/day]"], Q_sim)
            lmetric    = lmetric/cmetric

            # percentage change to higher values
            rsample    = params_calibrated.copy()
            rsample[k] = rsample[k]*(1 + r)
            Q_sim      = hbv(rsample, df_obs["P [mm/day]"], df_obs["T [C]"], df_obs["PET [mm/day]"])
            rsample    = rsample/csample
            rmetric    = [abs_bias, rmse][i](df_obs["Q [mm/day]"], Q_sim)
            rmetric    = rmetric/cmetric

            csample = csample/csample
            cmetric = cmetric/cmetric
            
            if param in ["CFMAX", "BETA", "FC", "K2"]:
                
                # index of the parameter value
                j = ["CFMAX", "BETA", "FC", "K2"].index(param)
                
                ax = fig.add_subplot(gsl[i + 1, j])
                if j == 0: ax.set_ylabel(f"Deviation of {metric}")
                if j != 0: ax.set_yticks([0.8, 1, 1.2], [])
                if j == 0: ax.set_yticks([0.8, 1, 1.2], ["-20%", "0%", "+20%"])
                ax.set_ylim([0.8, 1.2])
                ax.set_xlabel(f"Deviation from {param}")
                ax.set_xlim(0.8, 1.2)
                ax.set_xticks([0.8, 1.0, 1.2], ["-20%", "0%", "+20%"])
                
                #ax.axvline(csample[k], color="gray")
                ax.scatter(csample[k], cmetric, color="gray", label="Calibrated")
                ax.scatter(lsample[k], lmetric, c=f"C{k}", label="OAT")
                ax.scatter(rsample[k], rmetric, c=f"C{k}")
                ax.plot([lsample[k], csample[k], rsample[k]], [lmetric, cmetric, rmetric], color=f"C{k}", zorder=0)
                
                if j == 3: ax.legend()

            meanderiv = 0.5*abs(rmetric - cmetric)/(rsample[k] - csample[k]) + 0.5*abs(cmetric - lmetric)/(csample[k] - lsample[k])
            ax_sens.bar(k - 0.17 if i == 0 else k + 0.17, meanderiv, width=0.3, color=f"C{k}", hatch=None if i == 0 else "//", edgecolor="white", label=None if param != "BETA" else ["|Bias|", "RMSE"][i])
            
    

    ax_sens.legend()    
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.1, description='$\\Delta x$', max=0.2, min=0.01, readout_format='.0%…

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
    <h4><span>&#129300 </span>Task II: Local Sensitivity Analysis</h4>
    <p>
        We now take a step back and look at a more local approach. Around the calibrated model (e.g. the paramter set that has lowest metrics), we explore the paramter space using one-at-a-time sampling. This means that we only alter one parameter at a time, with a relative step size that you are free to set. We do this in both directions, so that for each parameter, we have three metric values: the step to lower values, the calibrated value and the step to higher values. They are drawn in the lower two rows of the plot, again once for RMSE and once for |Bias|.
    </p>
    <p>
        We can now use the average partial derivative around the calibrated value as a measure for the importance of the parameter. In other words: If a paramter is important, its choice should change the metric and therefore result in a higher derivative. In the first row of the plot, we calculated this derivative for both parameters and metrics.
    </p>
    <ol>
        <li>What are benefits and shortcomings to define "importance" using this approach?</li>
        <li>Which parameters are important? Are there differences between metrics?</li>
        <li>How does the step size influence parameter importance? How would you choose a limit for screening important paramters?</li>
    </ol>
</div>

_PUT YOUR ANSWER HERE_

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
</div>